In [1]:
# 판다스와 넘파이
import pandas as pd
import numpy as np

# 이미지
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#머신러닝
import lightgbm as lgb
from lightgbm import LGBMRegressor

# 검증 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 기타
from tqdm import tqdm
import warnings ; warnings.filterwarnings('ignore')

In [2]:
# train 데이터, date 컬럼 datetime 데이터로 변경
train = pd.read_csv(r'C:\phj\2\train.csv', encoding='cp949')
train['date_time'] = pd.to_datetime(train['date_time'])

# test 데이터, date 컬럼 datetime 데이터로 변경
test = pd.read_csv(r'C:\phj\2\test.csv', encoding='cp949')
test['date_time'] = pd.to_datetime(test['date_time'])

In [3]:
train.head()

,num,date_time,temperature,windspeed(m/s),humidity(%),precipitation(mm),sunshine(hr),non_electric_cooling_system,solar_cell,consumption(kWh)
0,1,2020-06-01 00:00:00,17.6,2.5,92.0,0.8,0.0,0,0,8179.056
1,1,2020-06-01 01:00:00,17.7,2.9,91.0,0.3,0.0,0,0,8135.640
2,1,2020-06-01 02:00:00,17.5,3.2,91.0,0.0,0.0,0,0,8107.128
3,1,2020-06-01 03:00:00,17.1,3.2,91.0,0.0,0.0,0,0,8048.808
4,1,2020-06-01 04:00:00,17.0,3.3,92.0,0.0,0.0,0,0,8043.624


In [4]:
test.head()

,num,date_time,temperature,windspeed(m/s),humidity(%),precipitation(mm),sunshine(hr),non_electric_cooling_system,solar_cell
0,1,2020-06-01 06:00:00,16.7,3.4,90.0,0.1,0.0,0,0
1,1,2020-06-01 11:00:00,22.1,3.6,52.0,0.0,1.0,0,0
2,1,2020-06-01 13:00:00,23.1,5.1,42.0,0.0,1.0,0,0
3,1,2020-06-02 02:00:00,15.9,1.1,78.0,0.0,0.0,0,0
4,1,2020-06-02 04:00:00,15.5,0.9,79.0,0.0,0.0,0,0


In [5]:
train.drop_duplicates(subset = ['num', 'non_electric_cooling_system', 'solar_cell']).dropna()

,num,date_time,temperature,windspeed(m/s),humidity(%),precipitation(mm),sunshine(hr),non_electric_cooling_system,solar_cell,consumption(kWh)
0,1,2020-06-01 00:00:00,17.6,2.5,92.0,0.8,0.0,0,0,8179.056
1632,2,2020-06-01 01:00:00,18.8,2.3,89.0,0.0,0.0,1,0,966.168
3264,3,2020-06-01 00:00:00,17.0,2.0,100.0,1.1,0.0,1,1,3183.624
4896,4,2020-06-01 00:00:00,22.6,0.7,62.0,0.0,0.0,1,1,339.552
6528,5,2020-06-01 00:00:00,18.3,2.5,94.0,0.0,0.0,1,0,3063.744
8160,6,2020-06-01 01:00:00,17.2,2.3,99.0,0.1,0.0,0,0,518.940
9792,7,2020-06-01 01:00:00,17.7,2.9,91.0,0.3,0.0,1,0,1298.592
11424,8,2020-06-01 00:00:00,19.1,1.1,73.0,0.0,0.0,1,1,4801.977
13056,9,2020-06-01 00:00:00,20.3,1.1,74.0,0.0,0.0,0,1,1255.932
14688,10,2020-06-01 00:00:00,18.3,2.5,94.0,0.0,0.0,1,0,417.960


In [6]:
X = train.drop([ 'date_time','consumption(kWh)'], axis = 1)
Y = train['consumption(kWh)']
X_test = test.drop('date_time', axis = 1)
         

In [7]:
X.head()

,num,temperature,windspeed(m/s),humidity(%),precipitation(mm),sunshine(hr),non_electric_cooling_system,solar_cell
0,1,17.6,2.5,92.0,0.8,0.0,0,0
1,1,17.7,2.9,91.0,0.3,0.0,0,0
2,1,17.5,3.2,91.0,0.0,0.0,0,0
3,1,17.1,3.2,91.0,0.0,0.0,0,0
4,1,17.0,3.3,92.0,0.0,0.0,0,0


In [14]:
n_splits = 10
sfk =StratifiedKFold(n_splits = n_splits)

y_pred =np.zeros(len(X_test))

for tr_idx, val_idx in sfk.split(X, X['num']):      
    X_train =X.iloc[tr_idx,:]
    Y_train =Y[tr_idx]

    X_val =X.iloc[val_idx,:]
    Y_val =Y[val_idx]

    lgb =LGBMRegressor(n_estimators = 10000, learning_rate= 0.1,objective = 'mse')
    lgb.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], early_stopping_rounds= 50,verbose = 100,eval_metric= 'mbo')
    y_pred +=lgb.predict(X_test) / n_splits

Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 356155
[200]	valid_0's l2: 353514
Early stopping, best iteration is:
[235]	valid_0's l2: 351844
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 515982
[200]	valid_0's l2: 508055
[300]	valid_0's l2: 507169
Early stopping, best iteration is:
[285]	valid_0's l2: 505974
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 615973
[200]	valid_0's l2: 610686
Early stopping, best iteration is:
[160]	valid_0's l2: 610212
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 555158
Early stopping, best iteration is:
[103]	valid_0's l2: 554533
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 581740
Early stopping, best iteration is:
[87]	valid_0's l2: 579694
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 580480
[200]	valid_0's l2: 566918
[300]	valid_0's l2: 564893
E